In [2]:
import sys
sys.path.append("/kaggle/working/sample/examples")
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from ultralytics import YOLO
import os
import glob
from tensorflow.python.ops import math_ops
tf.config.run_functions_eagerly(True)

In [3]:
real_US_annotations = "C:/Users/KIIT/OneDrive/Desktop/Vedant_Official/vedant projects and works/Datasets/Organ_segmentation_dataset/abdominal_US/abdominal_US/RUS/annotations"
real_US_Images = "C:/Users/KIIT/OneDrive/Desktop/Vedant_Official/vedant projects and works/Datasets/Organ_segmentation_dataset/abdominal_US/abdominal_US/RUS/images"
artificial_US_annotations = "C:/Users/KIIT/OneDrive/Desktop/Vedant_Official/vedant projects and works/Datasets/Organ_segmentation_dataset/abdominal_US/abdominal_US/AUS/annotations"
artificial_US_Images = "C:/Users/KIIT/OneDrive/Desktop/Vedant_Official/vedant projects and works/Datasets/Organ_segmentation_dataset/abdominal_US/abdominal_US/AUS/images"

In [5]:
def load_image_data(image_path,size=None, rgb=True):
    img = Image.open(image_path.decode())
    if not (size is None):
        img = img.resize(size.tolist())
    if not rgb:
        img = img.convert("L")
    else:
        img = img.convert("RGB")   
    img_data = np.array(img).astype(np.uint8)
    img.close()
    return img_data

In [6]:
def convert_color(rgb_arr, mode='rgb_to_str'):
  if mode == 'rgb_to_str':
    return np.apply_along_axis(','.join, 1, rgb_arr.astype(str))
  elif mode == 'str_to_rgb':
    return np.array([list(map(int, val.split(","))) for val in rgb_arr.split("_")[0].split(",")]).flatten()
  else:
    raise ValueError("Invalid mode. Supported modes are 'rgb_to_str' and 'str_to_rgb'.")

In [7]:
def generate_segment_dict():
    # violet = liver         yellow = kidney 
    # blue = pancreas        red = vessels
    # light blue= adrenals   green = gallbladder
    # white = bones          pink = spleen 
    # black = None
    seg_dict = {
        '0,0,0_none': 0,
        '100,0,100_liver': 1,
        '255,255,255_bone': 2,
        '0,255,0_gallbladder': 3,
        '255,255,0_kidney ': 4,
        '0,0,255_pancreas': 5,
        '255,0,0_vessels': 6,
        '255,0,255_spleen': 7,
        '0,255,255_adrenal': 8
    }
    return seg_dict

In [8]:
class SegmentDistances:
    def __init__(self, seg_dict):
        self.segment_arrs = np.array([convert_color(val,'str_to_rgb') for val in seg_dict.keys()])
        self.lookup = seg_dict
    
    def get_closest_segment(self, seg_img_arr):
        distances = np.linalg.norm(self.segment_arrs - seg_img_arr.astype(np.float16), axis=1)
        min_idx = np.argmin(distances)
        return min_idx
    
    def segments_to_sparse(self,seg_img):
        closest_segments = np.apply_along_axis(self.get_closest_segment, axis=2, arr=seg_img)
        closest_segments = closest_segments.astype(np.uint8)
        kernel = np.ones((5, 5), np.uint8) 
        img_erosion = cv2.erode(closest_segments, kernel, iterations=5) 
        img_dilation = cv2.dilate(img_erosion, kernel, iterations=3)
        return img_dilation
    
    def segments_to_masks(self,seg_sparse_img):
        height,width = seg_sparse_img.shape
        masks = np.zeros((height,width,len(self.lookup)-1))
        kernel = np.ones((5, 5), np.uint8) 

        for i in range(1,len(self.lookup)):
            masks[:,:,i-1] = (seg_sparse_img==i).astype(np.uint8)
            masks[:,:,i-1] = cv2.erode(masks[:,:,i-1], kernel, iterations=5) 
            masks[:,:,i-1] = cv2.dilate(masks[:,:,i-1], kernel, iterations=3)
        return masks
    
    def _preprocess_images(self,filename):
        out = tf.numpy_function(load_image_data,[filename,tf.constant((256,256))], tf.uint8)
#         out.set_shape((256,256,3))
        return out

    def _preprocess_annotations(self,filename):
        img = self._preprocess_images(filename)
        out = tf.numpy_function(self.segments_to_sparse,[img], tf.uint8)
#         out.set_shape((1,256,256))
        return out

In [21]:
seg_dicts = generate_segment_dict()
seg_dicts

{'0,0,0_none': 0,
 '100,0,100_liver': 1,
 '255,255,255_bone': 2,
 '0,255,0_gallbladder': 3,
 '255,255,0_kidney ': 4,
 '0,0,255_pancreas': 5,
 '255,0,0_vessels': 6,
 '255,0,255_spleen': 7,
 '0,255,255_adrenal': 8}

In [10]:
seg_dicts = generate_segment_dict()
SD = SegmentDistances(seg_dicts)

def make_data_set(path,f):
    files = glob.glob(path)
    file_data = tf.data.Dataset.from_tensor_slices(files)
    processed_data = file_data.map(f)
    return processed_data

tf_train_data = make_data_set(artificial_US_Images + "/train/*",SD._preprocess_images)
tf_test_data = make_data_set(artificial_US_Images + "/test/*",SD._preprocess_images)

tf_test_real_data = make_data_set(real_US_Images + "/test/*",SD._preprocess_images)


tf_train_annot = make_data_set(artificial_US_annotations + "/train/*",SD._preprocess_annotations)
tf_test_annot = make_data_set(artificial_US_annotations + "/test/*",SD._preprocess_annotations)

tf_test_real_annot = make_data_set(real_US_annotations + "/test/*",SD._preprocess_annotations)


tf_dataset = tf.data.Dataset.zip((tf_train_data, tf_train_annot))
tf_test = tf.data.Dataset.zip((tf_test_data, tf_test_annot))
tf_test_real = tf.data.Dataset.zip((tf_test_real_data, tf_test_real_annot))
# def set_shapes(image, label):
#     image.set_shape((256, 256, 3))
#     label.set_shape((256, 256,1))
#     return image, label
                    
# tf_dataset = tf_dataset.map(set_shapes)

c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [22]:
print(len(tf_train_data),len(tf_test_data),len(tf_test_real_data))
print(len(tf_train_annot),len(tf_test_annot),len(tf_test_real_annot))

633 293 213
633 293 61


In [31]:
def create_unet_model(input_shape, OUTPUT_CLASSES):
    def unet_model(output_channels:int):
        inputs = tf.keras.layers.Input(shape=input_shape)
#         inputs = tf.keras.layers.Rescaling(1/255.0)(inputs)
        # Downsampling through the model
        skips = down_stack(inputs)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])

        # This is the last layer of the UNET model
        x = tf.keras.layers.Conv2DTranspose(filters=100,
                                            kernel_size=3,
                                            strides=2,
                                            padding='same')(x)
        # Extra layer after the UNET to help with smoothing the output
        x = tf.keras.layers.Conv2D(filters=OUTPUT_CLASSES,
                                   kernel_size=3,
                                   strides=1,
                                   padding='same',
                                   activation="softmax"
                                   )(x)
        return tf.keras.Model(inputs=inputs, outputs=x)

    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False)
    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',
        'block_3_expand_relu',
        'block_6_expand_relu',
        'block_13_expand_relu',
        'block_16_project',
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)
    down_stack.trainable = False

    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3),  # 16x16 -> 32x32
        pix2pix.upsample(64, 3),   # 32x32 -> 64x64
    ]

    return unet_model(output_channels=OUTPUT_CLASSES)

In [32]:
model = create_unet_model([256, 256, 3], 9) # UNET Model
model.compile(optimizer='adam',
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
model.summary()

9406464/9406464 [==============================] - 3s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 model_2 (Functional)        [(None, 128, 128, 96),       1841984   ['input_11[0][0]']            
                              (None, 64, 64, 144),                                                
                              (None, 32, 32, 192),                                                
                              (None, 16, 16, 576),                                                
                              (None, 8, 8, 320)]                                                  
                             

In [24]:
from keras import backend as K
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [15]:
imgs,annot = [],[]
i = 0
for img,an in tf_dataset.as_numpy_iterator():
    if i%50==0:
        print(i)
    imgs.append(img)
    annot.append(an)
    i+=1
# test_out = [[img,annot] for img,annot in x.take(1).as_numpy_iterator()]
imgs = np.array(imgs)
annot = np.array(annot)

0
50
100
150
200
250
300
350
400
450
500
550
600


In [33]:
# model.fit(x=imgs,y=annot, epochs=10, batch_size=16)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping( patience=3, verbose=1, restore_best_weights=True)
model = model.fit(x=imgs, y=annot, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=10, 
                    # validation_data=(X_test, y_test_cat), 
                    # class_weight=class_weights,
                    shuffle=False,
                    callbacks=[early_stopping])
                    

Epoch 1/10


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(



40/40 [==============================] - 437s 11s/step - loss: 0.6403 - accuracy: 0.8313
Epoch 2/10
40/40 [==============================] - 497s 12s/step - loss: 0.3519 - accuracy: 0.8836
Epoch 3/10
40/40 [==============================] - 449s 11s/step - loss: 0.3154 - accuracy: 0.8947
Epoch 4/10
40/40 [==============================] - 465s 12s/step - loss: 0.2948 - accuracy: 0.9035
Epoch 5/10
40/40 [==============================] - 582s 15s/step - loss: 0.2838 - accuracy: 0.9074
Epoch 6/10
40/40 [==============================] - 953s 24s/step - loss: 0.2784 - accuracy: 0.9095
Epoch 7/10
40/40 [==============================] - 573s 14s/step - loss: 0.2717 - accuracy: 0.9123
Epoch 8/10
40/40 [==============================] - 412s 10s/step - loss: 0.2673 - accuracy: 0.9139
Epoch 9/10
40/40 [==============================] - 418s 10s/step - loss: 0.2634 - accuracy: 0.9143
Epoch 10/10
40/40 [==============================] - 339s 8s/step - loss: 0.2610 - accuracy: 0.9155


In [ ]:
model.save("best.keras")

In [ ]:
model = tf.keras.models.load_model('best.keras')

In [ ]:
model.evaluate(tf_test.batch(16))

In [ ]:
model.evaluate(tf_test_real.batch(16))

In [ ]:
from scipy.spatial import ConvexHull
seg_img = np.array([img for img in tf_train_annot.take(1).as_numpy_iterator()]).reshape((256,256))
classes = np.unique(seg_img.flatten())
with 
for cls in classes:
    print(classes)
    if cls == 0:
        continue
    mask = (seg_img==classes[1]).astype(np.uint8)
    value = np.where(mask)
    mask_points = np.array([[y,x] for y,x in zip(value[0],value[1])])
    ch = ConvexHull(mask_points)
    line_data = f'{cls} '
    for v in ch.vertices:
        point = mask_points[v] / 256
        line_data += " ".join(map(str,point))

In [ ]:
segmentation_model = YOLO("yolov8n-seg.pt")